
# RAG with Chroma

1. 문서 읽기
2. 문서를 청크로 나눈다.
3. 문서를 임베딩한다. -> 벡터 데이터베이스에 저장
4. 질문이 있을때 벡터데이터 베이스에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

## 1. 문서 읽기

In [59]:
%pip install -qU docx2txt langchain-community langchain-text-splitters unstructured python-docx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
#%pip install markdown

from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 읽기
loader = UnstructuredMarkdownLoader("./data/맞춤형복지_운영지침.md")

# chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
document_list = loader.load_and_split(text_splitter)


/Users/hunchulchoi/.pyenv/versions/3.12.12/envs/inflearn-llm-application/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
document_list


[Document(metadata={'source': './data/맞춤형복지_운영지침.md'}, page_content='📚 2023년도 전라북도교육감 소속 공무원 맞춤형복지제도 업무처리 세부지침\n\n[cite_start]시행일: 2023년 1월 1일부터 시행 [cite: 32] [cite_start]적용대상: 전라북도교육감 소속 공무원 및 사립학교 교직원 [cite: 34]\n\n📌 전년 대비 주요 변동 내용\n\n[cite_start]기본점수가 1,000점(300점 인상)으로 올랐습니다[cite: 19].\n\n[cite_start]건강검진비가 격년 350점(150점 인상)으로 오르고, 가족의 건강검진 지원이 가능해졌습니다[cite: 20].\n\n[cite_start]단체보험 선택항목에 수술비 보장, 배우자 생명·상해 보험, 배우자 및 자녀 의료비 보장 등이 신설되었습니다[cite: 21].\n\n[cite_start]단체보험 생명·상해 보험 가입금액 선택이 3천만원/5천만원/1억원/1.5억원/2억원으로 다양화되었습니다[cite: 21].\n\nⅠ. 맞춤형 복지제도 개요\n\n[cite_start]목적: 전라북도교육감 소속 공무원에 대한 맞춤형 복지제도 운영의 세부업무 처리기준을 정하여 공무원 후생복지업무의 효율성을 높이고자 함[cite: 25].\n\n[cite_start]복지점수 의미: 복지 혜택의 구매 및 정산에 사용되는 계산 단위로, 복지점수 1점은 1천원에 상당하는 것으로 봅니다[cite: 27, 95].\n\nⅡ. 맞춤형복지 항목 설계\n\n[cite_start]맞춤형 복지항목은 기본항목(필수/선택)과 자율항목으로 구성됩니다[cite: 45].\n\n1. 기본항목 (단체보험) 보장 내용'),
 Document(metadata={'source': './data/맞춤형복지_운영지침.md'}, page_content='[cite_start]복지점수 의미: 복지 혜택의 구매 및 정산에 사용되는 계산 단위로, 복지점수 1점은 1천원에 상당하는 것으로

## embedding
### sentence-transformers

In [42]:
%pip install sentence-transformers langchain-huggingface

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 16.7 MB/s eta 0:00:00a 0:00:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 25.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 14.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.4 MB/s eta 0:00:0

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

# embedding
embeddings = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')

## Chroma DB 생성후 파일로 저장

In [4]:

from langchain_chroma import Chroma

database = Chroma.from_documents(
    document_list, 
    embeddings,
    collection_name="gwp2003_markdown",
    persist_directory="./chroma_markdown_db",
    )

database    


## Chrmoma DB 로드

In [ ]:
from langchain_chroma import Chroma

database = Chroma(
    collection_name="gwp2003_markdown",
    persist_directory="./chroma_db",
    embedding_function=embeddings,
)

database 

NameError: name 'embeddings' is not defined

In [5]:
query = '보험 필수 항목은 무엇인가요?'

retd_docs = database.similarity_search(query)

retd_docs

[Document(id='5dc035d7-069f-4ca1-9d49-c689f11b29a6', metadata={'source': './data/맞춤형복지_운영지침.md'}, page_content='[cite_start]단체보험 계약 내용: 계약기간은 2023. 1. 1.부터 12. 31.까지 (1년)이며, 보험사는 현대해상화재보험(대표보험사) 등 3개 보험사 컨소시엄입니다[cite: 55, 56].\n\n2. 자율항목 구성 및 제한\n\n[cite_start]자율항목은 복지점수에서 단체보장보험 점수를 공제한 잔여 점수 범위 내에서 공무원이 자율적으로 선택 사용합니다[cite: 77].\n\n분야 자율항목(예시) 수혜대상 건강관리 [cite_start]병의원 진료, 약 구입, 안경 구입, 운동시설 이용 등 [cite: 81] [cite_start]본인 및 가족 [cite: 81] 자기계발 [cite_start]학원수강, 도서구입, 세미나 연수비 등 [cite: 81] [cite_start]본인 [cite: 81] 여가활용 [cite_start]여행 숙박시설, 레저시설 이용, 영화·연극관람 등 [cite: 81] [cite_start]본인 및 가족 [cite: 81] 가정친화 [cite_start]보육시설·노인복지시설 이용, 기념일 꽃 배달, 장례식 등 [cite: 81] [cite_start]가족 [cite: 81] 전통시장이용 [cite_start]전통시장상품권(온누리상품권) 등 구입 [cite: 81] [cite_start]본인 [cite: 81]\n\n청구 불가 항목:\n\n[cite_start]사행성이 있거나 불건전한 지출 (보석, 복권, 유흥비 등)[cite: 83].\n\n[cite_start]현금과 유사한 유가증권 및 물품 구매 (상품권, 주유권, 증권 등)[cite: 84].\n\n[cite_start]단, 전통시장상품권(온누리상품권)과 지역사랑상품권은 구매 가능[cite: 85].\n\n[cite_start]치료 목적이 아닌 미용 관련 의료행

In [6]:
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-2.5-flash")



In [7]:
#%pip install langchain_classic
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

In [8]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [14]:
#from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

retriever = database.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnableParallel(
        context=retriever,
        question=RunnablePassthrough(),    
    )
    | prompt
    | llm
)    

ai_message = rag_chain.invoke("맞춤형복지 제한 휴직 유형은 무엇인가요?")

print(ai_message)


제공된 컨텍스트에서는 맞춤형복지 제한을 받는 특정 휴직 유형에 대해 명시하고 있지 않습니다. 다만, 휴직 시 복지점수 사용 권한이 변동되어 월할 계산이 적용된다고 언급되어 있습니다. 변동복지점수 제한은 음주운전으로 견책 이상 징계를 받은 경우에 적용됩니다.


In [13]:
#%pip install -U langchain langchain-core langchain-community
%pip show langchain

Name: langchain
Version: 1.1.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /Users/hunchulchoi/.pyenv/versions/3.12.12/envs/inflearn-llm-application/lib/python3.12/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.
